In [1]:
import collections

from Bio import pairwise2
from Bio.Seq import Seq
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from data import AllBooks, Book
from utils import most_frequent

from align_config import dss_version, sp_version
from align_functions import make_alignments, collect_matching_words

from tf.app import use

MT = use('etcbc/bhsa')
Fmt, Tmt, Lmt = MT.api.F, MT.api.T, MT.api.L

SP = use('dt-ucph/sp:clone', checkout='clone', version=sp_version, provenanceSpec=dict(moduleSpecs=[]))
Fsp, Tsp, Lsp = SP.api.F, SP.api.T, SP.api.L

DSS = use('etcbc/dss', version=dss_version)
Fdss, Tdss, Ldss = DSS.api.F, DSS.api.T, DSS.api.L


C:\Users\geitb\anaconda3\envs\hebrew_spelling\Lib\site-packages\Bio\pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(
C:\Users\geitb\anaconda3\envs\hebrew_spelling\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,39,10938.21,100
chapter,929,459.19,100
lex,9230,46.22,100
verse,23213,18.38,100
half_verse,45179,9.44,100
sentence,63717,6.70,100
sentence_atom,64514,6.61,100
clause,88131,4.84,100
clause_atom,90704,4.70,100
phrase,253203,1.68,100


**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,5,79878.20,100
chapter,187,2135.78,100
verse,5841,68.38,100
word,114890,3.48,100
sign,399391,1.00,100


**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
scroll,1001,1428.81,100
lex,10450,129.14,94
fragment,11182,127.91,100
line,52895,27.04,100
clause,125,12.85,0
cluster,101099,6.68,47
phrase,315,5.10,0
word,500995,2.81,99
sign,1430241,1.00,100


In [2]:
SYLLABLE_TYPE = 'last'

MATRES_FILE = '../data/nouns_adjectives.csv'

In [3]:
PENTATEUCH_BOOKS = ['Genesis', 'Exodus', 'Leviticus', 'Numbers', 'Deuteronomy']
ALL_BOOK_NAMES = [Tmt.sectionFromNode(bo)[0] for bo in Fmt.otype.s('book')]

In [4]:
QSP_SCROLLS = {'1Qisaa', '1QisaaI', '1QisaaII', '2Q3', '4Q13', '4Q20', '2Q7', '4Q27', '1Q4', '2Q12', '4Q37', '4Q38', '4Q38a', '4Q40', '4Q53',
               '4Q57', '2Q13', '4Q78', '4Q80', '4Q82', '4Q128', '4Q129', '4Q134', '4Q135', '4Q136',
                '4Q137', '4Q138', '4Q139', '4Q140', '4Q141', '4Q142', '4Q143', '4Q144', '4Q158', '4Q364',
                '4Q365', '4Q96', '4Q111', '4Q109', '11Q5', '11Q6', '11Q7', '11Q8'}

# Prepare MT and SP texts

Produce dictionary mt_sp_matches which has mt words nodes as keys and matching word numbers from SP as values.

In [5]:
# prepare mt and sp books
MANUSCRIPTS = ['MT', 'SP']
all_books = AllBooks()
for book_name in ALL_BOOK_NAMES:
    book = Book('MT', book_name, Fmt, Tmt, Lmt)
    all_books.data[('MT', book_name)] = book
    
    if book_name in PENTATEUCH_BOOKS:
        book = Book('SP', book_name, Fsp, Tsp, Lsp)
        all_books.data[('SP', book_name)] = book

# Match words

In [6]:
dat = pd.read_csv(MATRES_FILE, sep='\t')
dat = dat[(dat.type == SYLLABLE_TYPE)] # & (dat.lex.isin(WAW_WORDS))]
print(dat.shape)

# dat_dss means non-MT, SP is included
dat_dss = dat[~dat.scroll.isin(['MT'])]
scroll_book_combinations = list(set(zip(dat_dss.scroll, dat_dss.book)))
dat_dss.shape

(20809, 37)


(6784, 37)

In [7]:
for manuscript, book_name in scroll_book_combinations:
    if manuscript == 'MT':
        continue
    elif manuscript == 'SP':
        book = Book(manuscript, book_name, Fsp, Tsp, Lsp)
        all_books.data[(manuscript, book_name)] = book
    else:
        book = Book(manuscript, book_name, Fdss, Tdss, Ldss)
        all_books.data[(manuscript, book_name)] = book

In [8]:
def make_matching_word_dict(book_name, all_books):

    all_match_dicts = {}

    matching_book = book_name
    matching_manuscripts = [scr for scr, bo in all_books.data.keys() if bo == matching_book]
    #print(matching_manuscripts)

    for man1 in matching_manuscripts:
        for man2 in matching_manuscripts:
            if man1 != man2:
            
                man1_verse_texts = all_books.data[(man1, matching_book)].verse_text_dict
                man2_verse_texts = all_books.data[(man2, matching_book)].verse_text_dict

                alignments_dict = make_alignments(man1_verse_texts, man2_verse_texts)
    
                man1_word2char = all_books.data[(man1, matching_book)].word2char
                man2_word2char = all_books.data[(man2, matching_book)].word2char
 
                matching_words_dict = collect_matching_words(alignments_dict, man1_word2char, man2_word2char)

                matches = {}
                for man1_word, man2_list in matching_words_dict.items():
                    man2_word = most_frequent(man2_list)
                    matches[man1_word] = man2_word
                all_match_dicts[((man1, matching_book), (man2, matching_book))] = matches
                
    return all_match_dicts, matching_manuscripts

In [9]:
def read_dataset(file):

    return pd.read_csv(file, sep='\t')

In [10]:
class MaterData:
    def __init__(self, man1, man2, section, lex, mater_val1, mater_val2, tf_id1, tf_id2, g_cons1, g_cons2):
        self.man1 = man1
        self.man2 = man2
        self.section = section
        self.lex = lex
        self.mater_val1 = mater_val1
        self.mater_val2 = mater_val2
        self.tf_id1 = tf_id1
        self.tf_id2 = tf_id2
        self.g_cons1 = g_cons1
        self.g_cons2 = g_cons2

In [11]:
def collect_matching_cases(matching_manuscripts, matching_book, dat):

    all_mater_datasets = {}

    for man in matching_manuscripts:
        mater_data = dat[(dat.book == matching_book) & (dat.scroll == man)]
        all_mater_datasets[(man, matching_book)] = mater_data
        
    return all_mater_datasets

In [12]:
def collect_mater_data(matching_manuscripts, all_match_dicts, all_mater_datasets, matching_book):

    manuscript_mater_match = collections.defaultdict(list)
    manuscripts = set()

    for idx, man in enumerate(matching_manuscripts):
        for idx2, man2 in enumerate(matching_manuscripts):
            if idx < idx2:
            
                matching_ids = all_match_dicts[((man, matching_book), (man2, matching_book))]
                man_data = all_mater_datasets[(man, matching_book)]
                man2_data = all_mater_datasets[(man2, matching_book)]
                for _, row in man_data.iterrows():
                
                    tf_id = row.tf_id
                    lex, typ, has_vl = row.lex, row.type, row.has_vowel_letter
                    g_cons1 = row.g_cons
                    section = (row.book, row.chapter, row.verse)
                    if man == 'SP':
                        tf_id = tf_id - 100000

                    matching_tf_id = matching_ids.get(tf_id, None)
                
                    if not matching_tf_id:
                        continue
                    
                    if man2 == 'SP':
                        matching_tf_id = matching_tf_id + 100000
                
                    man2_row = man2_data[(man2_data.tf_id == matching_tf_id) & (man2_data.lex == lex) & (man2_data.type == typ)]
                    
                    if not man2_row.shape[0]:
                        continue
                    has_vl2 = man2_row.has_vowel_letter.iloc[0]
                    g_cons2 = man2_row.g_cons.iloc[0]
                 
                    mater_data = MaterData(man, man2, section, lex, has_vl, has_vl2, tf_id, matching_tf_id, g_cons1, g_cons2)
                
                    manuscript_mater_match[man].append(mater_data)
                    manuscripts.add(man)
                    manuscripts.add(man2)
                    
    return manuscript_mater_match, manuscripts

In [13]:
def register_similarities_with_mt(manuscripts, mt_ids, manuscript_mater_match):

    mater_value_dict = {0: -1,
                        1: 1}

    mater_match_array = np.zeros((len(manuscripts), len(mt_ids)))

    for dat_object in manuscript_mater_match['MT']:
        
        other_man = dat_object.man2
        mt_tfid = dat_object.tf_id1
        other_man_tfid = dat_object.tf_id2
    
        mt_mater = dat_object.mater_val1
        other_man_mater = dat_object.mater_val2
    
        mt_idx = man2idx['MT']
        other_man_idx = man2idx[other_man]
    
        mt_mater_value = mater_value_dict[mt_mater]
        other_man_mater_value = mater_value_dict[other_man_mater]
    
        mt_tf_id = mt_tf2idx[mt_tfid]
    
        mater_match_array[mt_idx, mt_tf_id] = mt_mater_value
        mater_match_array[other_man_idx, mt_tf_id] = other_man_mater_value
        
    return mater_match_array

In [14]:
def get_parallels(manuscript_mater_match, hif_match_dict):
    for scroll in manuscript_mater_match.keys():
        for dat_object in manuscript_mater_match[scroll]:
            man1 = dat_object.man1
            man2 = dat_object.man2
            if man1 == 'MT' or man2 == 'MT':
            
                hif_match_dict[dat_object.tf_id2] = dat_object.tf_id1
                hif_match_dict[dat_object.tf_id1] = dat_object.tf_id2
        
    return hif_match_dict


In [15]:
def count_parallel_cases(mater_match_array):

    mater_arr = np.zeros((2, mater_match_array.shape[1]))

    for col_idx in range(mater_match_array.shape[1]):
        col = mater_match_array[:, col_idx]
        col_counts = collections.Counter(col)
        with_vowel_count = col_counts.get(1, 0)
        without_vowel_count = col_counts.get(-1, 0)
        mater_arr[0, col_idx] = with_vowel_count
        mater_arr[1, col_idx] = without_vowel_count
        
    return mater_arr

In [16]:
hif_match_dict = {}

for book in ALL_BOOK_NAMES:
    print(book)
    all_match_dicts, matching_manuscripts = make_matching_word_dict(book, all_books)
    all_mater_datasets = collect_matching_cases(matching_manuscripts, book, dat)
    manuscript_mater_match, manuscripts = collect_mater_data(matching_manuscripts, all_match_dicts, all_mater_datasets, book)
    
    hif_match_dict = get_parallels(manuscript_mater_match, hif_match_dict)
    print(len(hif_match_dict))
    

Genesis
1840
Exodus
3893
Leviticus
4810
Numbers
6134
Deuteronomy
8021
Joshua
8035
Judges
8039
1_Samuel
8127
2_Samuel
8233
1_Kings
8259
2_Kings
8259
Isaiah
10351
Jeremiah
10445
Ezekiel
10485
Hosea
10509
Joel
10553
Amos
10605
Obadiah
10621
Jonah
10694
Micah
10756
Nahum
10778
Habakkuk
10808
Zephaniah
10838
Haggai
10854
Zechariah
10864
Malachi
10868
Psalms
11426
Job
11444
Proverbs
11460
Ruth
11476
Song_of_songs
11501
Ecclesiastes
11509
Lamentations
11549
Esther
11549
Daniel
11611
Ezra
11611
Nehemiah
11611
1_Chronicles
11611
2_Chronicles
11611


In [17]:
all_scrolls = set(dat_dss.scroll) 
non_qsp_scrolls = all_scrolls.difference(QSP_SCROLLS)
non_qsp_data = dat_dss[dat_dss.scroll.isin(non_qsp_scrolls)]
non_qsp_pent = non_qsp_data[non_qsp_data.book.isin(PENTATEUCH_BOOKS)]
non_qsp_pent.shape

qsp_data = dat_dss[dat_dss.scroll.isin(QSP_SCROLLS)]

In [21]:
n = 0
full_count = collections.defaultdict(int)
scrolls = set()
scroll_counts = collections.defaultdict(int)


sp_mt_dict = collections.defaultdict(lambda: collections.defaultdict(int))

for index, row in non_qsp_data.iterrows():
    tf_id = row.tf_id, 
    scroll = row.scroll, 
    has_vl = row.has_vowel_letter
    book, ch, ve = row.book, row.chapter, row.verse
    g_cons = row.g_cons
    lex = row.lex
    prefix = row.prefix
    reconstr = row.rec_signs
    
    
    if tf_id[0] in hif_match_dict:
        corr_id = hif_match_dict[tf_id[0]]
        corr_row = dat[dat.tf_id == corr_id]
        #print(scroll)

        if lex != 'BRJX/':
            continue
        #if scroll[0] != 'SP':
        #    continue
        
        #print(scroll, corr_row.scroll.iloc[0], row.chapter, row.verse, row.lex, g_cons, corr_row.g_cons.iloc[0], 
        #      book, prefix, corr_row.prefix.iloc[0], has_vl, corr_row.has_vowel_letter.iloc[0])
        n += 1
        
        mt_full = corr_row.has_vowel_letter.iloc[0]
        scroll_full = has_vl
        full_count[(scroll_full, mt_full)] += 1
        scrolls.add(scroll)
        scroll_counts[scroll] += 1
        
        #if scroll[0] == '1Q8': # and lex[0] != 'J':
        #if has_vl != corr_row.has_vowel_letter.iloc[0]:
        print(scroll, has_vl, corr_row.has_vowel_letter.iloc[0], g_cons, corr_row.g_cons.iloc[0], prefix, reconstr, book, ch, ve)
            
            
    #else:
    #    print(scroll, book, row.chapter, row.verse, row.lex, row.g_cons)
    #    if scroll[0] != 'SP' and lex[0] == 'J':
    #           print(scroll, book, row.chapter, row.verse, row.lex, row.g_cons)
    #    continue
    
print(n)
print(full_count)
print(len((scrolls)))
print(scroll_counts)


('SP',) 1 1 BRJXJ BRJXM nan nnnnn Exodus 26 26
('SP',) 1 1 BRJXJM BRJXM nan nnnnnn Exodus 26 27
('SP',) 1 1 BRJXJM BRJXM nan nnnnnn Exodus 26 27
('SP',) 1 1 BRJX BRJX WH nnnn Exodus 26 28
('SP',) 1 1 BRJXJM BRJXM L nnnnnn Exodus 26 29
('SP',) 1 1 BRJXJM BRJXM H nnnnnn Exodus 26 29
('SP',) 1 1 BRJXJ BRJXJ nan nnnnn Exodus 36 31
('SP',) 1 1 BRJXJM BRJXM nan nnnnnn Exodus 36 32
('SP',) 1 1 BRJXJM BRJXM nan nnnnnn Exodus 36 32
('SP',) 1 1 BRJX BRJX H nnnn Exodus 36 33
('SP',) 1 1 BRJXJM BRJXM L nnnnnn Exodus 36 34
('SP',) 1 1 BRJXJM BRJXM H nnnnnn Exodus 36 34
('SP',) 1 1 BRJXJW BRJXJW nan nnnnnn Exodus 40 18
('SP',) 1 1 BRJXJW BRJXJW W nnnnnn Numbers 3 36
('SP',) 1 1 BRJXJW BRJXJW W nnnnnn Numbers 4 31
('SP',) 1 1 BRJX BRJX W nnnn Deuteronomy 3 5
('4Q17',) 1 1 BRJXJW BRJXJW nan nnnnnn Exodus 40 18
('4Q22',) 1 1 BRJXJ BRJXM nan nnnnn Exodus 26 26
('Mur88',) 0 0 BRXJH BRXJH nan rnnrr Jonah 2 7
19
defaultdict(<class 'int'>, {(1, np.int64(1)): 18, (0, np.int64(0)): 1})
4
defaultdict(<class 'i

In [24]:
dat[dat.nme == 'M']

,tf_id,scroll,book,chapter,verse,lex,g_cons,stem,pattern,pattern_g_cons,...,has_prs,has_prefix,has_hloc,has_nme,rec_signs_stem,cor_signs_stem,type,vowel_letter,has_vowel_letter,neigh_vowel_letter
130,1305,MT,Genesis,3,7,<JRM/,<JRMM,<JRM,CMCC,CMCCC,...,0,0,0,1,nnnn,nnnn,last,NaN,0,0
535,7676,MT,Genesis,17,20,NFJ>/,NFJ>M,NFJ>,CCMC,CCMCC,...,0,0,0,1,nnnn,nnnn,last,J,1,0
565,8208,MT,Genesis,18,24,YDJQ/,YDJQM,YDJQ,CCMC,CCMCC,...,0,0,0,1,nnnn,nnnn,last,J,1,0
567,8225,MT,Genesis,18,24,YDJQ/,YDJQM,YDJQ,CCMC,CCMCC,...,0,1,0,1,nnnn,nnnn,last,J,1,0
571,8269,MT,Genesis,18,26,YDJQ/,YDJQM,YDJQ,CCMC,CCMCC,...,0,0,0,1,nnnn,nnnn,last,J,1,0
574,8303,MT,Genesis,18,28,YDJQ/,YDJQM,YDJQ,CCMC,CCMCC,...,0,1,0,1,nnnn,nnnn,last,J,1,0
796,12666,MT,Genesis,25,16,NFJ>/,NFJ>M,NFJ>,CCMC,CCMCC,...,0,0,0,1,nnnn,nnnn,last,J,1,0
1361,22087,MT,Genesis,40,10,FRJG/,FRJGM,FRJG,CCMC,CCMCC,...,0,0,0,1,nnnn,nnnn,last,J,1,0
1938,32591,MT,Exodus,8,10,XMR=/,XMRM,XMR,CCC,CCCC,...,0,0,0,1,nnn,nnn,last,NaN,0,0
1939,32592,MT,Exodus,8,10,XMR=/,XMRM,XMR,CCC,CCCC,...,0,0,0,1,nnn,nnn,last,NaN,0,0


In [70]:
jbl_mtp = jbl[(jbl.scroll == 'MT') & (jbl.book.isin(PENTATEUCH_BOOKS))]
pd.crosstab(jbl_mtp.has_prefix, jbl_mtp.has_vowel_letter)

has_vowel_letter
has_prefix


In [71]:
jbl_mtp[jbl_mtp.has_prefix == 0]

,tf_id,scroll,book,chapter,verse,lex,g_cons,stem,pattern,pattern_g_cons,...,has_prs,has_prefix,has_hloc,has_nme,rec_signs_stem,cor_signs_stem,type,vowel_letter,has_vowel_letter,neigh_vowel_letter


In [72]:
jbl_fp = jbl[(jbl.scroll == 'MT') & (~jbl.book.isin(PENTATEUCH_BOOKS))]
pd.crosstab(jbl_fp.has_prefix, jbl_fp.has_vowel_letter)

has_vowel_letter
has_prefix


In [73]:
jbl_q = jbl[(jbl.scroll == '11Q1')]

In [21]:
collections.Counter(non_qsp_data.lex)

Counter({'R>CJT/': 25,
         '>LHJM/': 1245,
         'THWM/': 13,
         'MQWM/': 167,
         'CLJCJ/': 38,
         'M>WR/': 25,
         'GDWL/': 176,
         'RBJ<J/': 20,
         'XMJCJ/': 7,
         'JRQ/': 5,
         'CBJ<J/': 72,
         '<RWM/': 5,
         '<JRM/': 5,
         'XGWRH/': 1,
         'GXWN/': 2,
         'KRWB/': 26,
         'BKR/': 124,
         '<WN/': 77,
         'KNWR/': 8,
         'NXCT/': 64,
         '>XWT/': 50,
         'XBWRH/': 5,
         'CLC/': 217,
         'CMNH/': 34,
         'NPJLJM/': 3,
         'GBWR/': 21,
         'YDJQ/': 33,
         'VHR/': 82,
         'YPWR/': 25,
         '<FJRJ/': 11,
         'XLWN/': 5,
         'NJXX/': 46,
         'N<WRJM/': 12,
         'LCWN/': 18,
         'JBWSJ/': 19,
         '<RQJ/': 1,
         'GBWL/': 68,
         '>XRWN/': 18,
         'RKWC/': 14,
         '>LWN/': 7,
         'XMWR/': 56,
         '>TWN/': 20,
         'YPWN/': 31,
         'HR/': 8,
         'JLJD/': 12,
         